In [ ]:
{-# LANGUAGE MonomorphismRestriction #-}
{-# LANGUAGE KindSignatures #-}

import System.Random
import Control.Monad.State

In [ ]:
newtype MyState s a = MyState { runState :: s -> (a, s)}

returnState :: a -> MyState s a
returnState a = MyState $ \s -> (a, s)

bindState :: MyState s a -> (a -> MyState s b) -> MyState s b
bindState m k = MyState $ \s -> let (a, s') = runState m s
    in runState (k a) s'

getState :: MyState s s
getState = MyState $ \s -> (s, s)

putState :: s -> MyState s ()
putState s = MyState $ \_ -> ((), s)

In [ ]:
instance Functor (MyState s) where
    fmap f m = MyState $ \s -> let (a, s') = runState m s in (f a, s')

In [ ]:
:info Functor

In [ ]:
type RandomState = State StdGen

getRandom :: Random a => RandomState a
getRandom = do
    g <- get
    let (v, g') = random g
    put g'
    return v

getTwoRandoms :: Random a => RandomState (a, a)
getTwoRandoms = liftM2 (,) getRandom getRandom

In [ ]:
runTwoRandoms :: IO (Int, Int)
runTwoRandoms = do
    s <- getStdGen
    let (r, s') = runState getTwoRandoms s
    setStdGen s'
    return r

runTwoRandoms >>= print